# **Introduction to Verlet integration (exmaple: 2-body system)**

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

<p style="text-align: justify;font-size:15px"> 
In the molecular dyanmics simulations, the velocity, position and acceleration are sloved numerically by using the Verlet algorithm.
<p>

<p style="text-align: justify;font-size:20px"> 
$\mathbf{r}(t+\Delta t) = 2\mathbf{r}(t)-\mathbf{r}(t-\Delta t) + \mathbf{a}(t)\Delta t^2 + O(\Delta t^4)$
<p>

<p style="text-align: justify;font-size:20px">
$\mathbf{a}(t) = - \frac{1}{m}\nabla\mathbf{v}(\mathbf{r}(t))$
<p>
    
<p style="text-align: justify;font-size:23px">
$\mathbf{v}(t+ \Delta t) = \frac{\mathbf{r}(t+\Delta t)-\mathbf{r}(t-\Delta t)}{2\Delta t}$
<p>
    
<p style="text-align: justify;font-size:15px"> 
Since this is a numerical method, the choosing of the time step $\Delta t$ is very important. 
    If one chooses a very small time step to get accurate results, the simulations will take 
    long time to run. On the other hand, if one chooses a very large time step, the molecular 
    simulations will go to wrong results.
<p>
    
<p style="text-align: justify;font-size:15px"> 
Here, we domonstrates a two-body system (sun and earth) with the Verlet algorithm. The sun is 
    fixed at the centre positon. By tunning the time step with the slider, one can see how the
    choosing of the time step $\Delta t$ influences the trajectory of the earth.
<p>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from ipywidgets import Button, IntSlider, Output, VBox, HBox, Layout
from numpy.linalg import norm
import matplotlib.gridspec as gridspec

In [ ]:
def compute_force(r1, r2, m1, m2):
    x = G*m1*m2/np.linalg.norm(r1-r2)**2
    v = (r2-r1)/np.linalg.norm(r2-r1)
    return x*v

In [ ]:
def compute_acceleration(f, m):
    return f/m

In [ ]:
def verlet_update(r1, r2, v, m1, m2, dt, a):
    
    rn = 2.0*r1[-1] - r1[-2]  + dt*dt*a
    vn = (rn - r1[-2])/(2*dt)
    
    r1[-2] = r1[-1]
    r1[-1] = rn
    
    f = compute_force(rn, r2, m1, m2)
    a = compute_acceleration(f, m1)
    
    return r1, vn, a

def compute_energies(r1, r2, m1, m2, v):
    pot = -G*m1*m2/np.linalg.norm(r1-r2)
    kint = 0.5*m1*np.linalg.norm(v)**2
    angmom = np.cross(r1-r2, m1*v)
    
    return pot, kint, angmom

In [ ]:
%matplotlib widget

m1 = 1.0
m2 = 1000.0
G = 0.2
v = [0.0, 3.0]
dt = 0.01

r1 = np.array([[10, 0],[10.0, 3.0*dt]])
r2 = np.array([0, 0])

f = compute_force(r1[-1], r2, m1, m2)
a = compute_acceleration(f, m1)

etot0 = -G*m1*m2/np.linalg.norm(r1[0]-r2) + 0.5*m1*np.linalg.norm(v)**2

output = Output()

with output:
    global fig, ax1, ax2, ax3, sun, earth, origin, avector, fvector, trace, title
    fig = plt.figure(tight_layout=True, figsize=(9,5))
    fig.canvas.header_visible = False
    gs = gridspec.GridSpec(9, 5)
    
    ax1 = fig.add_subplot(gs[:, 0:3])
    ax2 = fig.add_subplot(gs[0:4, 3:5])
    ax3 = fig.add_subplot(gs[5:9, 3:5])

    sun = ax1.scatter([0.0],[0.0], s=23**2, marker="o", linewidth=0, color='gold', label = 'Sun')
    earth, = ax1.plot([10.0],[0.0], 'r.', alpha = 1.0, markersize=15, label = 'Earth')
    origin = ax1.plot([10.0],[0.0], 'kx', alpha = 1.0, markersize=15, label = 'Origin point')
    avector = ax1.quiver([10.0], [0], [0], [0], color = ['blue'], units='inches', scale = 10.0, label="a")
    fvector = ax1.quiver([10.0], [0], [0], [0], color = ['gold'], units='inches', scale = 10.0, label="v")
    trace, = ax1.plot([10.0], [0.0], "k", linewidth=0.5)
    title = ax1.set_title("")

    ax1.legend(loc=2, fontsize=12)

    ax1.set_xlim(-11.0, 11.0)
    ax1.set_ylim(-11.0, 11.0)
    ax1.grid()
    
    lpot, = ax2.plot([], [], 'k-', label='pot')
    lkint, = ax2.plot([], [], 'c-', label='kint')
    ltot, = ax2.plot([],[], 'r-', label='tot')
    ax2.axhline(etot0, color='blue', ls='--', linewidth=0.5)
    
    ax2.set_xlim(0, 20.0)
    ax2.set_ylim(-95, 60)
    ax2.legend(loc=3, ncol=3)
    ax2.set_title("Energies as a function of time", fontsize = 10)
    
    lang, = ax3.plot([],[], 'r-')
    ax3.axhline(30.0, color='blue', ls='--', linewidth=0.5)

    ax3.set_title("Angular momentum as a function of time", fontsize = 10)
    ax3.set_xlabel("Time")
    ax3.set_xlim(0, 20.0)
    ax3.set_ylim(20, 40)


xt = [];
yt = [];

lim1 = 0;
lim2 = 20.0;

t0 = 0.0;
te = [];
epot = [];
ekint = [];
etot = [];
am = [];

def init():
    return (earth, avector, fvector, trace)

    
def update(i):
    global r1, v, a, xt, yt, epot, ekint, etot, te, t0, lim1, lim2, am
    r1, v, a = verlet_update(r1, r2, v, m1, m2, dt, a)
    e1, e2, e3 = compute_energies(r1[-1,:], r2, m1, m2, v)

    earth.set_data([r1[-1, 0]], [r1[-1, 1]])
    xt.append(r1[-1, 0])
    yt.append(r1[-1, 1])
    
    epot.append(e1)
    ekint.append(e2)
    etot.append(e1+e2)
    am.append(e3)
    
    t0 = t0 + dt
    te.append(t0)
    
    if te[-1] > 20.0:
        lim1 = lim1 + dt
        lim2 = lim2 + dt
        ax2.set_xlim(lim1, lim2)
        ax3.set_xlim(lim1, lim2)

    lpot.set_data([te, epot])
    lkint.set_data([te, ekint])
    ltot.set_data([te, etot])
    lang.set_data([te, am])
    
    trace.set_data(xt, yt)
    avector.set_offsets(np.array([r1[-1, 0], r1[-1, 1]]))
    fvector.set_offsets(np.array([r1[-1, 0], r1[-1, 1]]))
    avector.set_UVC(a[0], a[1])
    fvector.set_UVC(v[0], v[1])
    
    title.set_text("v(t) = %-5.3f" % norm(v))

    return (earth, avector, fvector, trace, lpot, lang)


anim = FuncAnimation(fig, update, frames = 20000, interval = 10,
                    init_func=init, blit=True, repeat = True)


def onClick(event):
    if button_pause.description == "Pause":
        button_pause.description = "Play"
        anim.event_source.stop()
    else:
        button_pause.description = "Pause"
        anim.event_source.start()
        

def clear_data(b):
    global xt, yt
    
    xt = [];
    yt = [];

button_pause = Button(description="Pause");
button_pause.on_click(onClick)
button_clear = Button(description="Clear trace")
button_clear.on_click(clear_data)

plt.show()

def on_dt_change(c):
    global dt, r1, xt, yt
    
    anim.event_source.stop()
    
    dt = w_dt.value*0.01
    r1 = np.array([[10, 0],[10.0, 3.0*dt]])
    
    xt = []
    yt = []
    
    anim.event_source.start()

w_dt = IntSlider(value = 1, min = 1, max = 50, step=1, description="dt: ", orientation='horizontal',
                layout = Layout(width="400px"))
w_dt.observe(on_dt_change, names='value')

display(output, w_dt, button_pause, button_clear)